In [1]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
from scipy import ndimage

In [2]:
def array_from_text(text):
    arr = np.array([
        list(line.strip()) for line in text.strip().splitlines()
    ])
    return (arr == "#").astype(np.uint8)

def text_from_arr(arr):
    return "\n".join(
        "".join(
            "#" if x else "." for x in line
        ) for line in arr
    )


text = """
....#
#..#.
#..##
..#..
#....
""".strip()
arr = array_from_text(text)
assert text_from_arr(arr) == text

In [3]:
def sum_adjacent(arr):
    k = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
    return ndimage.convolve(arr, k, mode='constant', cval=0)

def step(arr):
    adj = sum_adjacent(arr)
    bug = arr.astype(np.bool)
    bug_survives = bug & (adj == 1)
    infested = ~bug & ((adj == 1) | (adj == 2))
    return (bug_survives | infested).astype(np.uint8)

def biodiversity(arr):
    return np.sum(arr.ravel() << np.arange(arr.size))

def simulate(arr):
    while True:
        yield arr
        arr = step(arr)

### Part 1

In [4]:
def run_until_dupe(arr):
    seen = set()
    for i, curr in tqdm(enumerate(simulate(arr))):
        x = biodiversity(curr)
        if x in seen:
            return curr, i, x
        seen.add(x)

In [5]:
arr = array_from_text(text)
_, _, x = run_until_dupe(arr)
assert x == 2129920

0it [00:00, ?it/s]


In [6]:
text = Path("input.txt").read_text()
arr = array_from_text(text)
run_until_dupe(arr)

0it [00:00, ?it/s]


(array([[0, 0, 1, 1, 1],
        [0, 0, 1, 1, 0],
        [0, 0, 1, 1, 0],
        [0, 0, 1, 1, 0],
        [1, 1, 0, 1, 1]], dtype=uint8), 19, 28717468)